## House Prices Prediction

### Competition Link:
https://www.kaggle.com/c/house-prices-advanced-regression-techniques

### Problem Statement:
To predict the sale prices of the houses given the features of the location and the house

### Data 
Dataset is called Ames Housing dataset compiled by dean de cock in kaggle

**Link**:https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

### Evaluation
The Kaggle vealuates our submission on the basis of Root-Mean-Squared-Error (RMSE)

**Link**:https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/evaluation

### Features
To know more about the data there is a data dictionary available which was also compiled by Dean de Cockunder the data fields heading

**Link**:https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data


In [1]:
# Importing all the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Getting the data ready

In [2]:
# Creating a dataframe 
df=pd.read_csv("../input/dean-de-cock-house-prices/train.csv",low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [3]:
# Checking for missing values
df.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [4]:
# Creating a copy of the dataframe so the edits doesn't affect the original dataframe
df_tmp=df.copy()

In [5]:
df_tmp.head().T

,0,1,2,3,4
Id,1,2,3,4,5
MSSubClass,60,20,60,70,60
MSZoning,RL,RL,RL,RL,RL
LotFrontage,65,80,68,60,84
LotArea,8450,9600,11250,9550,14260
...,...,...,...,...,...
MoSold,2,5,9,2,12
YrSold,2008,2007,2008,2006,2008
SaleType,WD,WD,WD,WD,WD
SaleCondition,Normal,Normal,Normal,Abnorml,Normal


## Filling the missing values in the df_tmp data

In [6]:
df_tmp.LotFrontage

0       65.0
1       80.0
2       68.0
3       60.0
4       84.0
        ... 
1455    62.0
1456    85.0
1457    66.0
1458    68.0
1459    75.0
Name: LotFrontage, Length: 1460, dtype: float64

### Coverting the strings features into pandas categorical features

In [7]:
# Checking the columns that contain strings
for label,content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

MSZoning
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
SaleType
SaleCondition


In [8]:
# Converting all the string data features into pandas categorical features
for label,content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label]=content.astype("category").cat.as_ordered()
        

In [9]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             1460 non-null   int64   
 1   MSSubClass     1460 non-null   int64   
 2   MSZoning       1460 non-null   category
 3   LotFrontage    1201 non-null   float64 
 4   LotArea        1460 non-null   int64   
 5   Street         1460 non-null   category
 6   Alley          91 non-null     category
 7   LotShape       1460 non-null   category
 8   LandContour    1460 non-null   category
 9   Utilities      1460 non-null   category
 10  LotConfig      1460 non-null   category
 11  LandSlope      1460 non-null   category
 12  Neighborhood   1460 non-null   category
 13  Condition1     1460 non-null   category
 14  Condition2     1460 non-null   category
 15  BldgType       1460 non-null   category
 16  HouseStyle     1460 non-null   category
 17  OverallQual    1460 non-null   in

In [10]:
df_tmp.MiscFeature.cat.categories

Index(['Gar2', 'Othr', 'Shed', 'TenC'], dtype='object')

In [11]:
df_tmp.MiscFeature.cat.codes

0      -1
1      -1
2      -1
3      -1
4      -1
       ..
1455   -1
1456   -1
1457    2
1458   -1
1459   -1
Length: 1460, dtype: int8

### Checking for missing values in the numeric columns

In [12]:
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

LotFrontage
MasVnrArea
GarageYrBlt


In [13]:
# Fill the numeric columns with median as median is so robust than mean
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            #Adding a binary column to tell if the data was missing
            df_tmp[label+"_is_missing"]=pd.isnull(content)
            #Filling the missing values with the median
            df_tmp[label]=content.fillna(content.median())
            

In [14]:
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

### As the above for loop did not print any output it is clear that all the numerical columns are filled 

In [15]:
for label,content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

Alley
MasVnrType
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Electrical
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PoolQC
Fence
MiscFeature


### **Note:** As we saw before cat.codes has -1 values due to the missing values but our model doesn't accept negative values so cat.codes+1 will make it 0

In [16]:

for label,content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
            #Adding a column to let us know what we are missing
            df_tmp[label+"_is_missing"]=pd.isnull(content)
            #Turn categories into number and add +1 to the codes
            df_tmp[label]=pd.Categorical(content).codes+1
            #Check the note above to understand why I'm using codes+1

In [17]:
# Checking to make sure that all the missing values in categories are filled
for label,content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                print(label)

In [18]:
# Checking the categorical codes of MiscFeature column
pd.Categorical(df_tmp["MiscFeature"]).codes

array([0, 0, 0, ..., 3, 0, 0], dtype=int8)

In [19]:
df_tmp.isna().sum()

Id                          0
MSSubClass                  0
MSZoning                    0
LotFrontage                 0
LotArea                     0
                           ..
PoolQC_is_missing           0
Fence_is_missing            0
MiscFeature_is_missing      0
SaleType_is_missing         0
SaleCondition_is_missing    0
Length: 127, dtype: int64

## Building the Model

In [20]:
#Building our baseline model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [21]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 127 entries, Id to SaleCondition_is_missing
dtypes: bool(46), float64(3), int64(35), int8(43)
memory usage: 560.5 KB


In [22]:
%%time
model=RandomForestRegressor(n_jobs=-1,random_state=69)


X=df_tmp.drop("SalePrice",axis=1)
y=df_tmp["SalePrice"]
#Fit the model
model.fit(X,y)


CPU times: user 4.18 s, sys: 42.5 ms, total: 4.23 s
Wall time: 1.21 s


RandomForestRegressor(n_jobs=-1, random_state=69)

In [23]:
model.score(X,y)

0.9813900625565077

### Building a evaluation function (RMSE) 

In [24]:
from sklearn.metrics import mean_squared_error
def rmsle(y_true,y_preds):
    return np.sqrt(mean_squared_error(y_true,y_preds))
def show_scores(model,X,y):
    y_preds=model.predict(X)
    scores={"Root Mean Squared Error":rmsle(y,y_preds)}
    return scores

In [25]:
show_scores(model,X,y)

{'Root Mean Squared Error': 10833.694708763096}

## Hyperparameter tuning using RandomizedSearch CV

In [26]:
%%time 

from sklearn.model_selection import RandomizedSearchCV
# Tuning the hyper parameters of the model
rs_grid={
    "n_estimators":np.arange(10,100,10),
    "max_depth":[None,3,5,10],
    "min_samples_split":np.arange(2,20,2),
    "min_samples_leaf":np.arange(1,20,2),
    "max_features":[0.5,1,"sqrt","auto"]
}

rs_model=RandomizedSearchCV(RandomForestRegressor(),param_distributions=rs_grid,n_iter=20,cv=5,verbose=True,random_state=69)

rs_model.fit(X,y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   25.9s finished


CPU times: user 26.5 s, sys: 70.6 ms, total: 26.6 s
Wall time: 26.6 s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=20,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_features': [0.5, 1, 'sqrt',
                                                         'auto'],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
                   random_state=69, verbose=True)

In [27]:
rs_model.best_params_

{'n_estimators': 60,
 'min_samples_split': 8,
 'min_samples_leaf': 1,
 'max_features': 0.5,
 'max_depth': 10}

### Training the model with the best parameters

In [28]:
ideal_model=RandomForestRegressor(n_estimators=60,min_samples_split=8,min_samples_leaf=1,max_features=0.5,max_depth=10,n_jobs=-1,random_state=69)

In [29]:
ideal_model.fit(X,y)

RandomForestRegressor(max_depth=10, max_features=0.5, min_samples_split=8,
                      n_estimators=60, n_jobs=-1, random_state=69)

In [30]:
X.shape

(1460, 126)

### Creating a dataframe for the testdata

In [31]:
test_df=pd.read_csv("../input/dean-de-cock-house-prices/test.csv")
test_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [32]:
test_df.shape

(1459, 80)

### Preprocessing the dataframe in the same format as training data

In [33]:
# Checking for the missing numeric features
for label,content in test_df.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

LotFrontage
MasVnrArea
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
BsmtFullBath
BsmtHalfBath
GarageYrBlt
GarageCars
GarageArea


In [34]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [35]:
#Checking for the non numeric features
for label,content in test_df.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

MSZoning
Alley
Utilities
Exterior1st
Exterior2nd
MasVnrType
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PoolQC
Fence
MiscFeature
SaleType


In [36]:
# Filling the numeric feature with median
for label,content in test_df.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            #Add a binary column to know which row is missing
            test_df[label+"_is_missing"]=pd.isnull(content)
            #Filling the missing content with median
            test_df[label]=content.fillna(content.median())

In [37]:
for label,items in test_df.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content):
            print(label)
            

In [38]:
#Convert the non numeric features into pandas categorical features
for label,content in test_df.items():
    if not pd.api.types.is_numeric_dtype(content):
        test_df[label+"_is_missing"]=pd.isnull(content)
        test_df[label]=pd.Categorical(content).codes+1

In [39]:
for label,content in test_df.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [40]:
test_df.shape

(1459, 134)

### Sometimes the training dataframe doesn't have the same attributes like the test dataframe so checking if there are any

In [41]:
missing=set(test_df.columns)-set(X.columns)
missing=list(missing)

In [42]:
set(test_df.columns)-set(X.columns)

{'BsmtFinSF1_is_missing',
 'BsmtFinSF2_is_missing',
 'BsmtFullBath_is_missing',
 'BsmtHalfBath_is_missing',
 'BsmtUnfSF_is_missing',
 'GarageArea_is_missing',
 'GarageCars_is_missing',
 'TotalBsmtSF_is_missing'}

These above attributes are not in the test dataframe

### Filling the above attributes in our test dataframe

In [43]:
for i in range(len(missing)):
    X[str(missing[i])]=False

In [44]:
set(X.columns)-set(test_df.columns)

set()

Now Test dataframe has same attributes as Training dataframe

In [45]:
test_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities',
       ...
       'GarageType_is_missing', 'GarageFinish_is_missing',
       'GarageQual_is_missing', 'GarageCond_is_missing',
       'PavedDrive_is_missing', 'PoolQC_is_missing', 'Fence_is_missing',
       'MiscFeature_is_missing', 'SaleType_is_missing',
       'SaleCondition_is_missing'],
      dtype='object', length=134)

### Making predictions on the test dataset

In [46]:
ideal_model.fit(X,y)

RandomForestRegressor(max_depth=10, max_features=0.5, min_samples_split=8,
                      n_estimators=60, n_jobs=-1, random_state=69)

In [47]:
y_preds=ideal_model.predict(test_df)

In [48]:
y_preds.max()


538264.6010951085

In [49]:
X.shape,test_df.shape,y.shape

((1460, 134), (1459, 134), (1460,))

### Preparing a dataframe for submission

In [50]:
# Making the dataframe in the format that kaggle asks for
sub_df=pd.DataFrame()
sub_df["Id"]=test_df["Id"]
sub_df["SalePrice"]=y_preds

In [51]:
sub_df.head()

,Id,SalePrice
0,1461,125779.002650
1,1462,153155.490700
2,1463,180717.190388
3,1464,187086.684638
4,1465,198419.543561


In [52]:
sub_df.to_csv("/kaggle/working/subfile.csv",index=False)

### Trying gradient boosting algorithm

In [53]:
from catboost import CatBoostRegressor

In [54]:
cat =CatBoostRegressor(random_state=69)
cat.fit(X,y)

Learning rate set to 0.04175
0:	learn: 77245.0942350	total: 58.9ms	remaining: 58.8s
1:	learn: 75225.8703381	total: 63.5ms	remaining: 31.7s
2:	learn: 73283.5063448	total: 67.9ms	remaining: 22.6s
3:	learn: 71552.5286141	total: 72.3ms	remaining: 18s
4:	learn: 69817.9856896	total: 76.4ms	remaining: 15.2s
5:	learn: 68041.1338987	total: 80.6ms	remaining: 13.4s
6:	learn: 66372.9797518	total: 84.6ms	remaining: 12s
7:	learn: 64714.4480342	total: 89ms	remaining: 11s
8:	learn: 63252.7352752	total: 93.1ms	remaining: 10.3s
9:	learn: 61703.2194489	total: 97.4ms	remaining: 9.64s
10:	learn: 60368.2841986	total: 102ms	remaining: 9.13s
11:	learn: 58903.2811458	total: 106ms	remaining: 8.69s
12:	learn: 57571.7428939	total: 110ms	remaining: 8.33s
13:	learn: 56462.7824820	total: 114ms	remaining: 8.03s
14:	learn: 55219.9113198	total: 118ms	remaining: 7.76s
15:	learn: 54036.6811646	total: 122ms	remaining: 7.52s
16:	learn: 52854.1710117	total: 126ms	remaining: 7.3s
17:	learn: 51950.2854412	total: 130ms	remaini

In [55]:
#Predicting the test data with catboost
cat_preds=cat.predict(test_df)

In [56]:
cat_mean=cat_preds.mean()

In [57]:
# Saving the baseline's prediction mean
randomf_mean=y_preds.mean()

In [58]:
# Checking how well our catboost's predictions are
randomf_mean-cat_mean

1002.0378971896425

In [59]:
boosted_csv=pd.DataFrame()
boosted_csv["Id"]=test_df["Id"]
boosted_csv["SalePrice"]=cat_preds

In [60]:
boosted_csv.head()

,Id,SalePrice
0,1461,125975.847845
1,1462,164296.750939
2,1463,183700.205241
3,1464,194370.247861
4,1465,182950.873688


In [61]:
boosted_csv.to_csv("/kaggle/working/boosted.csv",index=False)

In [62]:
cat_mean

177370.03575282206

### Tuning the Catboost Regressor

In [63]:
 model_CBR = CatBoostRegressor()

In [64]:
parameters = {'depth'         : [6,8,10],
                  'learning_rate' : [0.01, 0.05, 0.1],
                  'iterations'    : [30, 50]
                 }


In [65]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model_CBR, param_grid = parameters, cv = 2, n_jobs=-1)

In [66]:
grid.fit(X,y)

/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0:	learn: 74055.4090607	total: 7.25ms	remaining: 355ms
1:	learn: 69641.0885164	total: 14.1ms	remaining: 338ms
2:	learn: 65617.8190523	total: 20ms	remaining: 314ms
3:	learn: 61687.6694355	total: 25.6ms	remaining: 295ms
4:	learn: 58323.6611869	total: 31.2ms	remaining: 280ms
5:	learn: 55404.6005222	total: 37.1ms	remaining: 272ms
6:	learn: 52850.5208597	total: 43ms	remaining: 264ms
7:	learn: 50426.3820929	total: 47.9ms	remaining: 252ms
8:	learn: 47980.2723322	total: 53.6ms	remaining: 244ms
9:	learn: 45830.6274322	total: 59ms	remaining: 236ms
10:	learn: 44020.6736332	total: 64ms	remaining: 227ms
11:	learn: 42303.8219154	total: 69.7ms	remaining: 221ms
12:	learn: 40480.4814598	total: 74.8ms	remaining: 213ms
13:	learn: 38690.9095656	total: 80.1ms	remaining: 206ms
14:	learn: 37242.4170957	total: 85.7ms	remaining: 200ms
15:	learn: 36133.2707258	total: 91.4ms	remaining: 194ms
16:	learn: 35018.6201242	total: 96.4ms	remaining: 187ms
17:	learn: 34046.1190854	total: 102ms	remaining: 181ms
18:	learn: 

GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostRegressor object at 0x7efe27f92290>,
             n_jobs=-1,
             param_grid={'depth': [6, 8, 10], 'iterations': [30, 50],
                         'learning_rate': [0.01, 0.05, 0.1]})

In [67]:
grid.best_score_

0.8476259092861276

In [68]:
grid.best_params_

{'depth': 6, 'iterations': 50, 'learning_rate': 0.1}

In [69]:
tuned_cat=CatBoostRegressor(depth=6,iterations=100, learning_rate= 0.1)

In [70]:
tuned_cat.fit(X,y)

0:	learn: 74055.4090607	total: 4.6ms	remaining: 455ms
1:	learn: 69641.0885164	total: 9.64ms	remaining: 472ms
2:	learn: 65617.8190523	total: 13.9ms	remaining: 450ms
3:	learn: 61687.6694355	total: 18.1ms	remaining: 435ms
4:	learn: 58323.6611869	total: 22.4ms	remaining: 426ms
5:	learn: 55404.6005222	total: 27.2ms	remaining: 426ms
6:	learn: 52850.5208597	total: 31.8ms	remaining: 422ms
7:	learn: 50426.3820929	total: 36.2ms	remaining: 416ms
8:	learn: 47980.2723322	total: 41.4ms	remaining: 418ms
9:	learn: 45830.6274322	total: 46.3ms	remaining: 417ms
10:	learn: 44020.6736332	total: 51.3ms	remaining: 415ms
11:	learn: 42303.8219154	total: 56.1ms	remaining: 411ms
12:	learn: 40480.4814598	total: 60.4ms	remaining: 404ms
13:	learn: 38690.9095656	total: 65ms	remaining: 399ms
14:	learn: 37242.4170957	total: 69.3ms	remaining: 393ms
15:	learn: 36133.2707258	total: 73.7ms	remaining: 387ms
16:	learn: 35018.6201242	total: 77.8ms	remaining: 380ms
17:	learn: 34046.1190854	total: 82ms	remaining: 374ms
18:	lea

In [71]:
tuned_cat_predicts=tuned_cat.predict(test_df)

In [72]:
tuned_mean=tuned_cat_predicts.mean()

In [73]:
tuned_mean

177816.12675233363

### Since Checking the Tuned catboost performance was not satisfying now going on with Xgboost

In [74]:
import xgboost as xgb

In [75]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [76]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

### XG boost accepts the data as in the form of matrix 

In [77]:

train_x = X.values
test_x = test_df.values

In [78]:
xg_reg.fit(train_x,y)

[18:23:14] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[18:23:14] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=0, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=10, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [79]:
xg_predicts=xg_reg.predict(test_x)

In [80]:
xg_predicts.mean()

116343.04

In [81]:
xgboost_csv=pd.DataFrame()
xgboost_csv["Id"]=test_df["Id"]
xgboost_csv["SalePrice"]=xg_predicts

In [82]:
xgboost_csv.head()

,Id,SalePrice
0,1461,89080.070312
1,1462,96119.953125
2,1463,118751.937500
3,1464,120121.570312
4,1465,124036.039062


In [83]:
boosted_csv.head()

,Id,SalePrice
0,1461,125975.847845
1,1462,164296.750939
2,1463,183700.205241
3,1464,194370.247861
4,1465,182950.873688


In [84]:
xgboost_csv.to_csv("/kaggle/working/xgboosted.csv",index=False)

### Checking with the different set of tuning parameters

In [85]:
xgb1 = xgb.XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = -1,
                        verbose=True)



In [86]:
X_mat=X.values
test_mat=test_df.values

In [87]:
xgb_grid.fit(X_mat,
         y)

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   49.5s finished


[18:24:07] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[18:24:09] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=2,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=...
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parame

In [88]:
xgbtuned_predictions=xgb_grid.predict(test_mat)

In [89]:
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

0.8751992607203685
{'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'subsample': 0.7}


In [90]:
xgbtuned_predictions.mean()

177483.95

## This predictions was not better than the old xgboost predictions so I would recommend to stick with the older xgboost model